In [4]:
# !python -m pip install emoji --upgrade

  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49721 sha256=9cd96555cd3d6a03e39369b00b9deaa63abf0270852c953dc50c1a7999ca4125
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\4e\bf\6b\2e22b3708d14bf6384f862db539b044d6931bd6b14ad3c9adc
Successfully built emoji


In [10]:
import json
from main import preprocess
from tokenization import tokenize
from lemmatization import lemmatize

In [3]:
with open('meta.json', 'r', encoding='utf-8') as f:
    meta = json.load(f)
    
len(meta)

32

### Пример использования на 1 вхождении

In [7]:
test = 'Автор, какая у вас ох*енная собака! 😍💤😥'
tok = tokenize(test)

for i, params in meta.items():
    print(preprocess(tok, params=params))

автор , какая у вас охёенная собака ! 😍 💤 😥
автор , какая у вас охёенная собака ! 
автор , какая у вас охёенная собака ! emoji emoji emoji
автор , какая у вас охёенная собака ! pos_emoji neutral_emoji neg_emoji
автор , охёенная собака ! 😍 💤 😥
автор , охёенная собака ! 
автор , охёенная собака ! emoji emoji emoji
автор , охёенная собака ! pos_emoji neutral_emoji neg_emoji
автор , какой у вы охёенный собака ! 😍 💤 😥
автор , какой у вы охёенный собака ! 
автор , какой у вы охёенный собака ! emoji emoji emoji
автор , какой у вы охёенный собака ! pos_emoji neutral_emoji neg_emoji
автор , охёенный собака ! 😍 💤 😥
автор , охёенный собака ! 
автор , охёенный собака ! emoji emoji emoji
автор , охёенный собака ! pos_emoji neutral_emoji neg_emoji
автор какая у вас охёенная собака 😍 💤 😥
автор какая у вас охёенная собака 
автор какая у вас охёенная собака emoji emoji emoji
автор какая у вас охёенная собака pos_emoji neutral_emoji neg_emoji
автор охёенная собака 😍 💤 😥
автор охёенная собака 
автор охёе

## Пример использования на всех данных

In [11]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [17]:
filepath = 'example_data.tsv'
df = pd.read_csv(filepath, sep='\t')
df['tokenized'] = df['текст'].progress_apply(tokenize)
df['lemmatized'] = df['tokenized'].progress_apply(lemmatize)

In [18]:
df.head()

,Unnamed: 0,разметка,текст,контекст,community_id,community_type,tokenized,lemmatized
0,0,speech,Спасибо!,post,1,nontoxic,спасибо !,спасибо !
1,1,speech,вау спасибо большое!!,post,1,nontoxic,вау спасибо большое ! !,вау спасибо большой ! !
2,2,speech,Супер! Благодарю🙏,post,1,nontoxic,супер ! благодарю 🙏,супер ! благодарить 🙏
3,3,speech,"офигеть, спасибо!",post,1,nontoxic,"офигеть , спасибо !","офигеть , спасибо !"
4,4,speech,спасибо за проделанную работу 💙,post,1,nontoxic,спасибо за проделанную работу 💙,спасибо за проделать работа 💙


In [19]:
for i, params in tqdm(meta.items()):
    if params['lemmatization'] == 'no':
        df[i] = [preprocess(tok, params=params) for tok in tqdm(df['tokenized'])]
    if params['lemmatization'] == 'yes':
        params['lemmatization'] = 'no'
        df[i] = [preprocess(tok, params=params) for tok in tqdm(df['lemmatized'])]

In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [27]:
df.sample(10)

,разметка,текст,контекст,community_id,community_type,tokenized,lemmatized,0,1,2,...,22,23,24,25,26,27,28,29,30,31
5970,NaN,"[id557591534|Клод], мы продукт тех, кто нас сд...","[id407071736|Алексей], тогда по такой же логик...",3,toxic,"username , мы продукт тех , кто нас сделал . т...","username , мы продукт тот , кто мы сделать . т...","username , мы продукт тех , кто нас сделал . т...","username , мы продукт тех , кто нас сделал . т...","username , мы продукт тех , кто нас сделал . т...",...,username продукт тех сделал те сделал продукт ...,username продукт тех сделал те сделал продукт ...,username мы продукт тот кто мы сделать тот кто...,username мы продукт тот кто мы сделать тот кто...,username мы продукт тот кто мы сделать тот кто...,username мы продукт тот кто мы сделать тот кто...,username продукт сделать сделать продукт сдела...,username продукт сделать сделать продукт сдела...,username продукт сделать сделать продукт сдела...,username продукт сделать сделать продукт сдела...
5887,NaN,"А с возрастом понимаешь, что никто и не заметит",post,3,toxic,"а с возрастом понимаешь , что никто и не заметит","а с возраст понимать , что никто и не заметить","а с возрастом понимаешь , что никто и не заметит","а с возрастом понимаешь , что никто и не заметит","а с возрастом понимаешь , что никто и не заметит",...,возрастом понимаешь никто заметит,возрастом понимаешь никто заметит,а с возраст понимать что никто и не заметить,а с возраст понимать что никто и не заметить,а с возраст понимать что никто и не заметить,а с возраст понимать что никто и не заметить,возраст понимать никто заметить,возраст понимать никто заметить,возраст понимать никто заметить,возраст понимать никто заметить
10082,NaN,"[id33820910|Дмитрий], я подобрала сыну красиво...","Ну если отец оказался гнидой позорной, предал ...",4,toxic,"username , я подобрала сыну красивое отчество ...","username , я подобрать сын красивый отчество ,...","username , я подобрала сыну красивое отчество ...","username , я подобрала сыну красивое отчество ...","username , я подобрала сыну красивое отчество ...",...,username подобрала сыну красивое отчество звуч...,username подобрала сыну красивое отчество звуч...,username я подобрать сын красивый отчество что...,username я подобрать сын красивый отчество что...,username я подобрать сын красивый отчество что...,username я подобрать сын красивый отчество что...,username подобрать сын красивый отчество звуча...,username подобрать сын красивый отчество звуча...,username подобрать сын красивый отчество звуча...,username подобрать сын красивый отчество звуча...
6012,NaN,"[id19961032|Александр], есть и умершие","[id8531336|Антон], а есть заболевшие?",3,toxic,"username , есть и умершие","username , есть и умерший","username , есть и умершие","username , есть и умершие","username , есть и умершие",...,username умершие,username умершие,username есть и умерший,username есть и умерший,username есть и умерший,username есть и умерший,username умерший,username умерший,username умерший,username умерший
5875,NaN,"[id588219929|Алина], ты же не в красной книге,...","[id10183073|Влад], не обижайте животных",3,toxic,"username , ты же не в красной книге , чтобы та...","username , ты же не в красный книга , чтобы та...","username , ты же не в красной книге , чтобы та...","username , ты же не в красной книге , чтобы та...","username , ты же не в красной книге , чтобы та...",...,username красной книге таких животных охраняли,username красной книге таких животных охраняли,username ты же не в красный книга чтобы такой ...,username ты же не в красный книга чтобы такой ...,username ты же не в красный книга чтобы такой ...,username ты же не в красный книга чтобы такой ...,username красный книга животное охранять,username красный книга животное охранять,username красный книга животное охранять,username красный книга животное охранять
7592,NaN,И продолжают тереть..,post,4,toxic,и продолжают тереть . .,и продолжать тереть . .,и продолжают тереть . .,и продолжают тереть .

In [25]:
df.to_csv('example_preprocessed_data.tsv', sep='\t')

## Если нужно поменять опции то допишите дополнительные циклы сюда и в main.py соответственно

In [29]:
meta = {}

i = 0
for punct in ['no', 'yes']:
    for lem in ['no', 'yes']:
        for stop in ['no', 'yes']:
            for emo in ['no', 'del', 'replace', 'label']:
                meta[i] = {'punctuation_deletion': punct,
                           'lemmatization': lem,
                           'stopwords_deletion': stop, 
                           'emojis_processing': emo}
                i += 1
                
meta

{0: {'punctuation_deletion': 'no',
  'lemmatization': 'no',
  'stopwords_deletion': 'no',
  'emojis_processing': 'no'},
 1: {'punctuation_deletion': 'no',
  'lemmatization': 'no',
  'stopwords_deletion': 'no',
  'emojis_processing': 'del'},
 2: {'punctuation_deletion': 'no',
  'lemmatization': 'no',
  'stopwords_deletion': 'no',
  'emojis_processing': 'replace'},
 3: {'punctuation_deletion': 'no',
  'lemmatization': 'no',
  'stopwords_deletion': 'no',
  'emojis_processing': 'label'},
 4: {'punctuation_deletion': 'no',
  'lemmatization': 'no',
  'stopwords_deletion': 'yes',
  'emojis_processing': 'no'},
 5: {'punctuation_deletion': 'no',
  'lemmatization': 'no',
  'stopwords_deletion': 'yes',
  'emojis_processing': 'del'},
 6: {'punctuation_deletion': 'no',
  'lemmatization': 'no',
  'stopwords_deletion': 'yes',
  'emojis_processing': 'replace'},
 7: {'punctuation_deletion': 'no',
  'lemmatization': 'no',
  'stopwords_deletion': 'yes',
  'emojis_processing': 'label'},
 8: {'punctuation_

In [ ]:
with open('meta.json', 'w') as f:
    json.dump(meta, f, ensure_ascii=False)